In [1]:
import pandas as pd
import json

In [3]:
 # Open file with the raw data
file = open("data.json", encoding="utf8")

# Load data as JSON
raw_data = json.load(file)

# Close the original file
file.close()

In [4]:
# this is what the sample data looks like
raw_data

[{'id': '0001',
  'type': 'donut',
  'name': 'Cake',
  'ppu': 0.55,
  'batters': {'batter': [{'id': '1001', 'type': 'Regular'},
    {'id': '1002', 'type': 'Chocolate'},
    {'id': '1003', 'type': 'Blueberry'},
    {'id': '1004', 'type': "Devil's Food"}]},
  'topping': [{'id': '5001', 'type': 'None'},
   {'id': '5002', 'type': 'Glazed'},
   {'id': '5005', 'type': 'Sugar'},
   {'id': '5007', 'type': 'Powdered Sugar'},
   {'id': '5006', 'type': 'Chocolate with Sprinkles'},
   {'id': '5003', 'type': 'Chocolate'},
   {'id': '5004', 'type': 'Maple'}]},
 {'id': '0002',
  'type': 'donut',
  'name': 'Raised',
  'ppu': 0.55,
  'batters': {'batter': [{'id': '1001', 'type': 'Regular'}]},
  'topping': [{'id': '5001', 'type': 'None'},
   {'id': '5002', 'type': 'Glazed'},
   {'id': '5005', 'type': 'Sugar'},
   {'id': '5003', 'type': 'Chocolate'},
   {'id': '5004', 'type': 'Maple'}]},
 {'id': '0003',
  'type': 'donut',
  'name': 'Old Fashioned',
  'ppu': 0.55,
  'batters': {'batter': [{'id': '1001', '

In [5]:
# load it as a data frame
df = pd.DataFrame(raw_data)
df

,id,type,name,ppu,batters,topping
0,0001,donut,Cake,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."
1,0002,donut,Raised,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'}]}","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."
2,0003,donut,Old Fashioned,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."


In [6]:
# write function to convert dict to dot notation paths
# this will also add a sequential digit to differentiate similar filepaths

def walk_path(input_value, old='', count=None):
    """
    convert json to dot notation paths
    """
    if isinstance(input_value, dict):
        if input_value :# if a not an empty dictionary:
            for key in input_value.keys():
                
                walk_path(input_value[key], old+'.'+str(key))
        else:
            
            all_paths.append(old)
            all_vals.append('{}')
            
    elif isinstance(input_value, list):
        if input_value:
            for i, k in enumerate(input_value):
                walk_path(k, old+'.'+str(i))
        else:
            
            all_paths.append(old)
            all_vals.append('[]')
    else:
        all_paths.append(old)
        all_vals.append(str(input_value))
    
    return all_vals, all_paths
    

In [7]:
# test out the function on the 'batters' column
all_results = []
df['batters_reformated'] = ''

for index, row in df.iterrows():
    batters = row['batters']
    all_paths = []
    all_vals = []
    x, y = walk_path(batters)
    res =pd.DataFrame(x, y).reset_index()
    res.columns = ['dot_path', 'value']
    
    res_dict = dict(zip(res.dot_path, res.value))
    df.at[index, 'batters_reformated'] = res_dict
    all_results.append(res_dict)

In [8]:
df

,id,type,name,ppu,batters,topping,batters_reformated
0,0001,donut,Cake,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'...","{'.batter.0.id': '1001', '.batter.0.type': 'Re..."
1,0002,donut,Raised,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'}]}","[{'id': '5001', 'type': 'None'}, {'id': '5002'...","{'.batter.0.id': '1001', '.batter.0.type': 'Re..."
2,0003,donut,Old Fashioned,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'...","{'.batter.0.id': '1001', '.batter.0.type': 'Re..."


In [9]:
# original data for batters
df['batters'][0]

{'batter': [{'id': '1001', 'type': 'Regular'},
  {'id': '1002', 'type': 'Chocolate'},
  {'id': '1003', 'type': 'Blueberry'},
  {'id': '1004', 'type': "Devil's Food"}]}

In [10]:
# reformatted data based on my func 
df['batters_reformated'][0]

{'.batter.0.id': '1001',
 '.batter.0.type': 'Regular',
 '.batter.1.id': '1002',
 '.batter.1.type': 'Chocolate',
 '.batter.2.id': '1003',
 '.batter.2.type': 'Blueberry',
 '.batter.3.id': '1004',
 '.batter.3.type': "Devil's Food"}